In [1]:
import numpy as np
import math
from numpy.linalg import inv, svd

In [13]:
def fit(X, y, lmbda):
    cs = np.unique(y)
    U, s, V = svd(X, full_matrices = False)
    V = V.T
    Z = U.dot(np.diag(s))
    Sigma = np.cov(Z, rowvar = False, ddof = 0)
    Sigma = lmbda * np.diag(Sigma) + (1 - lmbda) * Sigma
    VInvSigma = V.dot(inv(Sigma))
    Betas = []
    
    for c in cs:
        i = y == c
        prior = np.mean(i)
        mu_c = np.mean(X[i], axis = 0)
        mu_z_c = V.T.dot(mu_c)
        Beta = VInvSigma.dot(mu_z_c)
        gamma = -mu_c.T.dot(Beta) / 2.0 + math.log(prior)
        Betas.append(np.hstack((Beta, gamma)))
        
    return cs, np.transpose(Betas)
        
def predict(model, X):
    cs, Beta = model
    N, D = X.shape
    C = len(cs)
    loglik = np.empty((N, C))
    
    X = np.hstack((X, np.ones((len(X), 1))))
    loglik = X.dot(Beta)
    
    return cs[np.argmax(loglik, axis = 1)]

In [24]:
X = np.array([
    [2, 3, 4, 5],
    [2.2, 3.1, 4.2, 5.3],
    [2.1, 3.2, 4.1, 5.2],
])

y = np.array([1, 1, 2])

In [26]:
model = fit(X, y, 0.0)
np.mean(predict(model, X) == y)

0.33333333333333331